In [3]:
%pip install -r requirements.txt

import sys
def in_venv():
    return sys.prefix != sys.base_prefix
print(in_venv())

from data import *
from models import *
from point_cloud_generator import *
from pointcloudhelpers import *
from utils import *
from helpers import *
from main import *
import pyvista as pv 
import numpy as np

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.
Defaulting to user installation because normal site-packages is not writeable
  Using cached bokeh-3.7.0-py3-none-any.whl (7.0 MB)
  Using cached Bottleneck-1.4.2-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (356 kB)
  Using cached Brotli-1.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.0 MB)
  Using cached cachecontrol-0.14.2-py3-none-any.whl (21 kB)
  Using cached cached_property-1.5.2-py2.py3-none-any.whl (7.6 kB)
  Using cached cachy-0.3.0-py2.py3-none-any.whl (20 kB)
  Using cached Cartopy-0.24.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.7 MB)
  Using cached click_default_group-1.2.4-py2.py3-none-any.whl (4.1 kB)
  Using cached clikit-0.6.2-py2.py3-none-any.whl (91 kB)
  Using cached cytoolz-1.0.1-cp310-cp31

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Load YAML config
config = load_config("configs/config.yaml")

# # Set random seed for reproducibility
seed_everything(config.seed, workers=True)
print("Config Loaded Successfully")

# # Instantiate DataModule with optimized settings
data_module = AtmosphereDataModule(config)

# # Instantiate Model
target_str = config.model._target_
model = INRModel(config).to(set_device())

print("Generating point cloud from trained model...")

# Load trained model from checkpoint
checkpoint_path = config.model_checkpoint  # Ensure this is defined in config
model.load_state_dict(torch.load(checkpoint_path, map_location="cpu"))
model.eval()

# Initialize PointCloudGenerator
config.model._target_ = target_str
pc_generator = PointCloudGenerator(model, config, device="cpu")

for render_type in RENDER_TYPES:
    render_type = render_type.lower()
    if render_type not in RENDER_TYPES:
        raise Exception(f"Invalid render type: must be {RENDER_TYPES}, got {render_type}")
    pointcloud_filename = pc_generator.generate(render_type=render_type)

    print(f"Point cloud saved to {pointcloud_filename}")